In [34]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

from download_PurpleAir_data import *
import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
city_latitude = 34.0522   # Replace with the latitude of the city's center
city_longitude = -118.2437  # Replace with the longitude of the city's center
radius = 10000  # Radius in meters

sensor_ids = get_purpleair_sensor_ids(city_latitude, city_longitude, radius)
print(sensor_ids)

TypeError: list indices must be integers or slices, not str

In [22]:
start_time = datetime.datetime(2024, 6, 1, 0, 0).timestamp() #1652156890
end_time = datetime.datetime(2024, 8, 4, 0, 0).timestamp()# 1675238400

df = fetch_purpleair_data(
    sensor_ids = [117579],
    start_time = start_time,
    end_time = end_time,
    path = './Data/Sacramento'
)

df

/Users/nicholasbachand/Documents/Research/PurpleAir/download_PurpleAir_data.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.read_csv(StringIO(response.text))])


44944 rows exported and saved


,time_stamp,sensor_index,pressure,temperature
0,1717225316,117579,100985.0,89.0
1,1717225436,117579,100981.0,89.0
2,1717225556,117579,100988.0,89.0
3,1717225676,117579,100987.0,89.0
4,1717225796,117579,100990.0,89.0
...,...,...,...,...
44939,1722639016,117579,101564.0,93.0
44940,1722639136,117579,101563.0,93.0
44941,1722639256,117579,101560.0,93.0
44942,1722639376,117579,101556.0,93.0
